In [ ]:
'''Extract Features'''
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
def extract_X():

    file_path = r""

    Columns=[]
    # C-J:data of sampling points at different depths
    for i in range(16):
        df_specific_sheet = pd.read_excel(file_path, sheet_name=i+1)
        # using sheet index (the first sheet serves as the table of contents)
        for c in range(8):
            column=df_specific_sheet.iloc[:, 2+c][1:]
            Columns.append(column.values)

    samples=[]
    for i in range(49):
        
        samples.append(i)
    X=[]
    for sample in samples:
        for depth in range(8):
            features=[]
            # print(depth)
            features.append(depth*10)
            for f in range(16):
                val=Columns[f*8+depth][sample]
                features.append(val)

            X.append(features)
    X=np.array(X)
    return X
X=extract_X()
print(X.shape)

In [ ]:
'''Extract Labels'''
import pandas as pd
import numpy as np
def extract_Y():
    # Reading data
    file_path = r""
    Columns=[]

    df_specific_sheet = pd.read_excel(file_path, sheet_name="Total UVAs") 
    for c in range(8):
        column=df_specific_sheet.iloc[:, 2+c][1:]
        Columns.append(column.values)

    Y=[]

    samples=[]
    for i in range(49):
        
        samples.append(i)
    for sample in samples:
        
        for depth in range(8):
            # 8列
            labels=[]

            for f in range(1):
                # f是1个sheet
                # 一个文件有8个cols
                val=Columns[f*8+depth][sample]
                labels.append(val)

            Y.append(labels)
                
    Y=np.array(Y)

    return Y

In [ ]:
# feature_names = [
#     "Depth",  #0      # depth
#     "Temp",   #1      # Temperature  
#     "Sal",    #2      # Salinity
#     "HFVel",  #3      # Horizontal Flow Velocity
#     "VFVel",  #4      # Vertical Flow Velocity
#     "DO",     #5      # Dissolved Oxygen
#     "DIC",    #6      # Dissolved Inorganic Carbon
#     "NO3",    #7      # Nitrate
#     "PO4",    #8      # Phosphate
#     "SiO4",   #9      # Silicate
#     "Fe2+",   #10      # Dissolved Iron Ions
#     "Chl",    #11      # Chlorophyll
#     "EHS",    #12
#     "OD-PABA",  #13
#     "EHMC",     #14 
#     "OC",       #15

# ]
def filter_data(X,Y):
    """
    filter_data: kill nonce data
    """
    Train_X=[]
    Train_Y=[]
    for i in range(len(Y)):
        label=0
        for m in range(X.shape[1]):
            if np.isnan(X[i][m]):
                label=1
                break
        if np.isnan(Y[i][0]) !=1 and label==0 and Y[i][0]<1000:
            Train_X.append(X[i])
            Train_Y.append(Y[i])

    X_Train=np.array(Train_X)
    Y_Train=np.array(Train_Y)
    cols=[]
    X_Train = np.delete(X_Train, cols, axis=1)
    print("Check shape:")
    print(X_Train.shape,Y_Train.shape)
    return X_Train,Y_Train

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from tqdm import *
import tensorflow as tf
from keras.models import Model
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.regularizers import *
from sklearn.model_selection import KFold
import random
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from keras.callbacks import EarlyStopping
from sklearn.ensemble import RandomForestRegressor

In [ ]:
def ResDual_mlp_model(input_size:int, learning_rate=0.004, key_feature_idx=-4):
    inputs = Input(shape=(input_size,))
    
    
    key_feature = inputs[:, key_feature_idx:]
    
    # channel 1
    x_key_main = Dense(4, kernel_regularizer=l2(0.01), name='key_fc1', activation=None,kernel_initializer='he_normal')(key_feature)
    x_key_main = BatchNormalization()(x_key_main)
    x_key_main = Dropout(0.05)(x_key_main)

    x_key_main = Dense(abs(key_feature_idx), kernel_regularizer=l2(0.01), activation=None,kernel_initializer='he_normal')(x_key_main)
    x_key_main = BatchNormalization()(x_key_main)
    x_key_main = Dropout(0.05)(x_key_main)

    x_key = Add()([x_key_main, key_feature])
    x_key = Activation('relu')(x_key)
    x_key = BatchNormalization()(x_key)
    x_key_main = Dropout(0.05)(x_key_main)


    # channel 2
    other_features = inputs[:, :key_feature_idx]
    x_other = Dense(8, activation='relu', kernel_regularizer=l2(0.05), name='other_fc1',kernel_initializer='he_normal')(other_features)
    x_other = BatchNormalization()(x_other)
    x_other = Dropout(0.05)(x_other)

    x_other = Dense(8, activation='relu', kernel_regularizer=l2(0.05), name='other_fc2',kernel_initializer='he_normal')(x_other)
    x_other = BatchNormalization()(x_other)
    x_other = Dropout(0.05)(x_other)


    # Merge
    merged = Concatenate()([x_key, x_other])
    x = Dense(128, activation='relu', name='merged_fc1',kernel_initializer='he_normal')(merged)
    x = BatchNormalization()(x)
    # 256 units is also acceptible
    outputs = Dense(1, activation='linear')(x)
    model = Model(inputs, outputs)
    
    optimizer = Adam(learning_rate=learning_rate, clipnorm=2.0)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# can use optuna for further optimization


In [ ]:



X=extract_X()
Y=extract_Y()

X_all,Y_all=filter_data(X,Y)


kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
fold_histories = {
    'train_loss': [], 'val_loss': [],
    'train_mae': [], 'val_mae': [],
    'test_mae': [], 'test_rmse': []
}


plt.figure(figsize=(12, 6))
data_size = len(X_all)
train_size = int(data_size * 0.8)  


import random
r=random.randint(0,256)
random.seed(42)
start_index = random.randint(0, data_size - train_size + 1)


X_Train, X_Test, y_Train, Y_test = train_test_split(X_all, Y_all, test_size=0.2, random_state=22)
scaler = StandardScaler()
for fold, (train_idx, test_idx) in enumerate(kf.split(X_Train)):
    
    y_scaler = StandardScaler()
    scaler = StandardScaler()
    X_train = scaler.fit_transform(np.log1p(X_Train[train_idx]))
    X_test = scaler.transform(np.log1p(X_Train[test_idx]))

    y_train = y_scaler.fit_transform(np.log1p(y_Train[train_idx]).reshape(-1, 1)).flatten()
    y_test = y_scaler.fit_transform(np.log1p(y_Train[test_idx]).reshape(-1, 1)).flatten()



    model = ResDual_mlp_model(16,key_feature_idx=-4)

    X_train_fold, X_val_fold, y_train_fold, y_val_fold = train_test_split(
        X_train, y_train, test_size=0.15, random_state=42
    )
    

    history = model.fit(
        X_train_fold, y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        epochs=100,
        batch_size=16,
        verbose=1,
        callbacks=[
            EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
        ]
    )
    
  
    best_epoch = np.argmin(history.history['val_loss'])
    fold_histories['train_loss'].append(history.history['loss'][best_epoch])
    fold_histories['val_loss'].append(history.history['val_loss'][best_epoch])
    fold_histories['train_mae'].append(history.history['mae'][best_epoch])
    fold_histories['val_mae'].append(history.history['val_mae'][best_epoch])
    

    y_pred = model.predict(X_test).flatten()
    fold_histories['test_mae'].append(mean_absolute_error(y_test, y_pred))
    fold_histories['test_rmse'].append(np.sqrt(mean_squared_error(y_test, y_pred)))
    

    plt.plot(history.history['val_mae'], alpha=0.3, linestyle='--', color=plt.cm.tab10(fold))



plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.show()


report = {
    'train_loss': f"{np.mean(fold_histories['train_loss']):.4f} ± {np.std(fold_histories['train_loss']):.4f}",
    'val_loss': f"{np.mean(fold_histories['val_loss']):.4f} ± {np.std(fold_histories['val_loss']):.4f}",
    'train_mae': f"{np.mean(fold_histories['train_mae']):.4f} ± {np.std(fold_histories['train_mae']):.4f}",
    'val_mae': f"{np.mean(fold_histories['val_mae']):.4f} ± {np.std(fold_histories['val_mae']):.4f}",
    'test_mae': f"{np.mean(fold_histories['test_mae']):.4f} ± {np.std(fold_histories['test_mae']):.4f}",
    'test_rmse': f"{np.mean(fold_histories['test_rmse']):.4f} ± {np.std(fold_histories['test_rmse']):.4f}"
}

In [ ]:
X_Test=scaler.transform(np.log1p(X_Test))
y_pred=np.expm1(y_scaler.inverse_transform(model.predict(X_Test)))
y_test=Y_test
print("Predict: ", list(np.round(y_pred.flatten(), 4)))

print("TrueVal: ", list(np.round(y_test.flatten(), 4)))

y_pred=y_pred.reshape(y_pred.shape[0],y_pred.shape[1])
import numpy as np
from scipy.stats import spearmanr
rho, p_value = spearmanr(y_pred.flatten(), y_test.flatten())

print(f"Spearman's rank correlation coefficient: {rho}")
print(f"P-value: {p_value}")

from scipy.stats import pearsonr
correlation, _ = pearsonr(y_pred.flatten(), y_test.flatten())


print(f"pearsonr correlation coefficient: {correlation:.4f}")
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)


print(f"Mean Absolute Error (MAE): {mae}")
from sklearn.metrics import mean_squared_error


rmse = np.sqrt(mean_squared_error(y_test, y_pred))


print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

from sklearn.metrics import r2_score
# 计算 R^2
r2 = r2_score(Y_test, y_pred)
# 打印 R^2
print(f"R_squared (R^2): {r2:.4f}")

In [ ]:
import shap
X_all = scaler.transform(X_all)

explainer = shap.DeepExplainer(model, X_train[:])  
shap_values = explainer.shap_values(X_train)  

feature_names = [
    "Depth",  #0      # depth
    "Temp",   #1      # Temperature  
    "Sal",    #2      # Salinity
    "HFVel",  #3      # Horizontal Flow Velocity
    "VFVel",  #4      # Vertical Flow Velocity
    "DO",     #5      # Dissolved Oxygen
    "DIC",    #6      # Dissolved Inorganic Carbon
    "NO3",    #7      # Nitrate
    "PO4",    #8      # Phosphate
    "SiO4",   #9      # Silicate
    "Fe2+",   #10      # Dissolved Iron Ions
    "Chl",    #11      # Chlorophyll
    "EHS",    #12
    "OD-PABA",  #13
    "EHMC",     #14 
    "OC",       #15

]

feature_importance = np.mean(np.abs(shap_values), axis=0).flatten()
sorted_idx = np.argsort(feature_importance)[::-1]

sorted_importance = feature_importance[sorted_idx]
print("SHAP:",sorted_importance)
sorted_names = [feature_names[i] for i in sorted_idx]
plt.figure(figsize=(12, 8))  
colors = plt.cm.viridis(sorted_importance / max(sorted_importance)) 
bars = plt.bar(range(len(sorted_importance)), sorted_importance, align='center', color=colors)

for i, v in enumerate(sorted_importance):
    plt.text(i, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=8)

plt.xticks(range(len(sorted_importance)), sorted_names, rotation=45, fontsize=15)
plt.xlabel("Features")
plt.ylabel("Mean |SHAP Value|")
plt.title("SHAP")
plt.tight_layout()


# 显示图表
plt.show()